In [83]:
# 这个notebook是用来把设计表里平铺的数据，散落/更新/添加/修改到项目的配置表里
# 这张表格是设计表： intermediateTables\PreCATFishesOrigin.xlsx
# 我们要使用它，来向 configs\xlsx_channel\2001\fish_env_affinity.xlsx 以及 configs\xlsx_channel\2001\fish_pond.xlsx 中添加/修改数据
# 数据应该是单向流动的，尽量不要修改configs目录下的表格
import os.path

rootPath = ".."
ORIGIN_TABLE = r"intermediateTables\PreCATFishesOrigin.xlsx"
AffinitySpreadSheet = r"configs\xlsx_channel\2001\fish_env_affinity.xlsx"
FishBasicSpreadSheet = r"configs\xlsx_channel\2001\fish_basic.xlsx"
PondSpreadSheet = r"configs\xlsx_channel\2001\fish_pond.xlsx"

ORIGIN_TABLE = os.path.join(rootPath, ORIGIN_TABLE)
AffinitySpreadSheet = os.path.join(rootPath, AffinitySpreadSheet)
PondSpreadSheet = os.path.join(rootPath, PondSpreadSheet)
FishBasicSpreadSheet = os.path.join(rootPath, FishBasicSpreadSheet)

sheetOrigin = "鱼种设计表"
sheetDictQuality = "品质字典"

startColumnNameStruct = "|调整后遮蔽"
startColumnNameBait = "|饵"
startColumnNameWaterLayer = "|觅食层"
startColumnNameTemp = "|温度"

columnNameFishSpeci = "Species"

dictTempColumns = {
    "中间温":"temperature_fav",
    "调整后温度耐受":"temp_affected_ratio",
    "温度耐受阈值":"temp_threshold",
    }

In [84]:
import pandas as pd

OriginDf = pd.read_excel(ORIGIN_TABLE, sheet_name="鱼种设计表")
# OriginDf.head()
OriginDf.columns

dictQualityDf = pd.read_excel(ORIGIN_TABLE, sheet_name=sheetDictQuality)
WeightParamDf = pd.read_excel(ORIGIN_TABLE, sheet_name="中英文和LW参数")


In [85]:
# 把columns切成几段，以那些|开头的为分割线
def split_columns(columns):
    split_columns = []
    current_segment = []
    for col in columns:
        # 强制转为字符串再判断前缀，避免非 str 类型（如 int）报错
        if str(col).startswith("|"):
            if current_segment:
                split_columns.append(current_segment)
            current_segment = [col]
        else:
            current_segment.append(col)
    if current_segment:
        split_columns.append(current_segment)
    return split_columns

# print the segments
splittedCols = split_columns(OriginDf.columns)
print(splittedCols)

for segment in splittedCols:
    print(segment)

[['Species', '鱼种', '是否保底', '等阶', '细化等阶', '备注', '本阶内单条价格', '品质', '地图', '阶内权重', 'xp参考', 'gold参考'], ['|温度', '舒适温度min', '舒适温度max', '中间温', '温度耐受', '温度耐受调整', '调整后温度耐受', '温度耐受阈值'], ['|遮蔽', '水下结构体]开放水域', '水下结构体]水草', '水下结构体]石头', '水下结构体]沉木', '水下结构体]桥墩'], ['|调整后遮蔽', '[水下结构体]开放水域', '[水下结构体]水草', '[水下结构体]石头', '[水下结构体]沉木', '[水下结构体]桥墩'], ['|觅食层', '[地图水层]表层', '[地图水层]中层', '[地图水层]底层'], ['|饵', '[真饵]昆虫', '[真饵]甲壳', '[真饵]鱼饵', '[真饵]鱼块', '[真饵]鱼卵', '[真饵]面团', '[真饵]谷物', '[拟饵]T尾', '[拟饵]卷尾', '[拟饵]软虫', '[拟饵]虾管', '[拟饵]米诺', '[拟饵]波爬', '[拟饵]勺子亮片', '[拟饵]旋转亮片', '[拟饵]VIB', '[拟饵]水面拖拉机', '[拟饵]铅笔'], ['|其他', '气压敏感度'], ['|推演水温亲和', '摄氏度下亲和度', 10, 12, 14, 16, 18, 20, 22, 24, 26]]
['Species', '鱼种', '是否保底', '等阶', '细化等阶', '备注', '本阶内单条价格', '品质', '地图', '阶内权重', 'xp参考', 'gold参考']
['|温度', '舒适温度min', '舒适温度max', '中间温', '温度耐受', '温度耐受调整', '调整后温度耐受', '温度耐受阈值']
['|遮蔽', '水下结构体]开放水域', '水下结构体]水草', '水下结构体]石头', '水下结构体]沉木', '水下结构体]桥墩']
['|调整后遮蔽', '[水下结构体]开放水域', '[水下结构体]水草', '[水下结构体]石头', '[水下结构体]沉木', '[水下结构体]桥墩']
['|觅食层', '[地图水层]表层', '[地图水层]中层', '[地图

In [86]:
# 从splittedCols里取出以startColumnNameStruct开头的segment,并去除startColumnNameStruct这一列
# 然后动态的遍历修改AffinitySpreadSheet中的数据
def get_segment_by_start_name(splittedCols, startName):
    for segment in splittedCols:
        if segment[0].startswith(startName):
            return segment[1:]
    return None

# 取出以|遮蔽开头的segment,并去除|遮蔽这一列
segmentStruct = get_segment_by_start_name(splittedCols, startColumnNameStruct)
print(segmentStruct)


['[水下结构体]开放水域', '[水下结构体]水草', '[水下结构体]石头', '[水下结构体]沉木', '[水下结构体]桥墩']


In [87]:
# 开始遍历修改AffinitySpreadSheet中的数据
import openpyxl

openpyxl.open(AffinitySpreadSheet, read_only=False, keep_links=False)
AffinityPyxl = openpyxl.load_workbook(AffinitySpreadSheet)
openpyxl.open(FishBasicSpreadSheet, read_only=False, keep_links=False)
FishBasicPyxl = openpyxl.load_workbook(FishBasicSpreadSheet)
# 温度sheet的名字是“TempAffinity”
tempAffinitySheetPath = AffinityPyxl["TempAffinity"]
structAffinitySheetPath = AffinityPyxl["StructAffinity"]
baitTypeAffinitySheetPath = AffinityPyxl["BaitTypeAffinity"]
topAffinitySheetPath = AffinityPyxl["FishEnvAffinity"]

topAffinitySheet = AffinityPyxl["FishEnvAffinity"]

fishQualitySheet = FishBasicPyxl["BasicFishQuality"]

fishNames = OriginDf[columnNameFishSpeci].tolist()
originRows = OriginDf.shape[0]


In [88]:
# fishQuality表的第二行是列名，第五行起是数据
# 先找到id为name，name_language， art_id， model_id， species， size_category， volume_exponent， mass_factor的几列的索引
id_col = None
fish_quality_name_col = None
name_language_col = None
art_id_col = None
model_id_col = None
species_col = None
size_category_col = None
volume_exponent_col = None
mass_factor_col = None
for col in range(1, fishQualitySheet.max_column + 1):
    cell = fishQualitySheet.cell(row=2, column=col)
    if cell.value == "id":
        id_col = col
    elif cell.value == "name":
        fish_quality_name_col = col
    elif cell.value == "name_language":
        name_language_col = col
    elif cell.value == "art_id":
        art_id_col = col
    elif cell.value == "model_id":
        model_id_col = col
    elif cell.value == "species":
        species_col = col
    elif cell.value == "size_category":
        size_category_col = col
    elif cell.value == "volume_exponent":
        volume_exponent_col = col
    elif cell.value == "mass_factor":
        mass_factor_col = col

In [ ]:


def getFishNameRowFromTopAffinitySheet(fishName,quality) -> int:
    qualityStr = dictQualityDf[dictQualityDf["品质"] == quality]["后缀"].values[0]
    prefixedFishSpeciesName = f"Fish_{fishName}"
    prefixedFishQualityName = f"Fish_{fishName}{qualityStr}"
    fishQualityName = f"{fishName}{qualityStr}"
    # 在表格中查找这个名字
    foundFishName = False
    # 在表格中B列查找这个名字
    for row in range(2, topAffinitySheet.max_row + 1):
        cell = topAffinitySheet.cell(row=row, column=2)
        if cell.value == prefixedFishQualityName:
            foundFishName = True
            return row
            break
    if not foundFishName:
        lastId = topAffinitySheet.cell(row=topAffinitySheet.max_row, column=1).value
        if lastId is None:
            lastId = 1010300
        else:
            newId = lastId + 10 - lastId % 10

        # add a new row, and set the fish name in B column
        new_row_idx = topAffinitySheet.max_row + 1
        topAffinitySheet.cell(row=new_row_idx, column=2, value=prefixedFishQualityName)
        topAffinitySheet.cell(row=new_row_idx, column=1, value=newId)
        # set the fish name in the 3rd column
        topAffinitySheet.cell(row=new_row_idx, column=3, value=fishQualityName)
        # from column 4, copy the values from the last row, and set this row font color to green
        for col in range(4, topAffinitySheet.max_column + 1):
            cell = topAffinitySheet.cell(row=new_row_idx, column=col)
            new_cell = topAffinitySheet.cell(row=new_row_idx, column=col)
            new_cell.value = cell.value
            new_cell.font = openpyxl.styles.Font(color="00FF00")

        # 检查fish_basic.xlsx中的fish_quality工作表中是否有鱼的这个品质配置，如果没有，进行相应的配置，并把相应的行设置成绿色
        foundQuality = False
        # fishQuality表的第二行是列名，第五行起是数据
        for rowIdx in range(5, fishQualitySheet.max_row + 1):
            cell = fishQualitySheet.cell(row=rowIdx, column=fish_quality_name_col)
            if cell.value == fishName:
                foundQuality = True
                break
        if not foundQuality:
            lastRowIdx = fishQualitySheet.max_row
            lastId = fishQualitySheet.cell(row=lastRowIdx, column=id_col).value
            newId = lastId + 10 - lastId % 10
            # 在fishQualitySheet中添加一行数据
            # create a new row by copying the last row
            new_row_idx = lastRowIdx + 1
            for col in range(1, fishQualitySheet.max_column + 1):
                cell = fishQualitySheet.cell(row=lastRowIdx, column=col)
                new_cell = fishQualitySheet.cell(row=new_row_idx, column=col)
                new_cell.value = cell.value
                new_cell.font = openpyxl.styles.Font(color="00FF00")
            # set the new row's id to newId
            fishQualitySheet.cell(row=new_row_idx, column=id_col, value=newId)
            # set the new row's name to fishName
            fishQualitySheet.cell(row=new_row_idx, column=fish_quality_name_col, value=fishQualityName)
            # lower case of prefixedFishSpeciesName
            lowerCasedPrefixedSpecies = prefixedFishSpeciesName.lower()
            # set the new row's name_language to name_language_value
            fishQualitySheet.cell(row=new_row_idx, column=name_language_col, value=lowerCasedPrefixedSpecies)
            # art_id
            fishQualitySheet.cell(row=new_row_idx, column=art_id_col, value=lowerCasedPrefixedSpecies+"_2D")
            fishQualitySheet.cell(row=new_row_idx, column=model_id_col, value=lowerCasedPrefixedSpecies)
            fishQualitySheet.cell(row=new_row_idx, column=species_col, value=lowerCasedPrefixedSpecies)
            sizeCategoryStr = dictQualityDf[dictQualityDf["品质"] == quality]["枚举"].values[0]
            fishQualitySheet.cell(row=new_row_idx, column=size_category_col, value=sizeCategoryStr)
            # volume_exponent
            volumeExponentValue = WeightParamDf[WeightParamDf["手调鱼种string"] == fishName]["重量参数b"].values[0]
            fishQualitySheet.cell(row=new_row_idx, column=volume_exponent_col, value=volumeExponentValue)
            # mass_factor
            massFactorValue = WeightParamDf[WeightParamDf["手调鱼种string"] == fishName]["重量参数a"].values[0]
            fishQualitySheet.cell(row=new_row_idx, column=mass_factor_col, value=massFactorValue)



        return new_row_idx
    return -1
        
for row in range(originRows):
    fishName = OriginDf.iloc[row][columnNameFishSpeci]
    # 强制把品质列转成字符串再 split，避免 int 没有 split
    raw_q = OriginDf.at[row, "品质"]
    qualities = str(raw_q).split("、")
    for q in qualities:
        # 如果后续需要用整数作 key，再尝试转回 int
        try:
            quality_key = int(q)
        except ValueError:
            quality_key = q
        getFishNameRowFromTopAffinitySheet(fishName, quality_key)

KeyError: '名称'

In [ ]:
def extract_qualities_for_fish(fish_name, fish_col=columnNameFishSpeci, quality_col="品质"):
    """根据鱼名从设计表中提取品质列表，转换为整数或保留原字符串"""
    # 查找对应行
    matches = OriginDf[OriginDf[fish_col] == fish_name]
    if matches.empty:
        return []
    raw_q = matches.iloc[0][quality_col]
    # 分割并转换
    qualities = []
    for q in str(raw_q).split("、"):
        try:
            qualities.append(int(q))
        except ValueError:
            qualities.append(q)
    return qualities

# 示例调用：
# print(extract_qualities_for_fish(OriginDf, 'Rock_Bass'))

## 水下结构体数据格式转换
将源表中的宽格式数据（每行一种鱼有多列不同结构体的数值）转换成目标表中的结构化格式（每行一种鱼，结构体类型和系数交替排列的多列）

In [ ]:
import pandas as pd
from openpyxl.utils.dataframe import dataframe_to_rows

# 定义宽格式转结构化格式的函数
def convert_wide_to_structured(source_df, struct_columns):
    """
    将源表中的宽格式数据转换为目标表中的结构化格式
    
    参数:
    - source_df: 源DataFrame，包含鱼类和结构体系数
    - struct_columns: 水下结构体列名列表
    
    返回:
    - 转换后的DataFrame，适合写入目标表格
    """
    # 获取鱼种名称列 (如果存在)
    fish_name_col = 'name' if 'name' in source_df.columns else 'Species'
    
    # 准备结果DataFrame
    result_rows = []
    
    # 为每个鱼种创建一行
    for idx, row in source_df.iterrows():
        # 开始构建新行
        new_row = {}
        
        # 添加ID和鱼名 (如果存在)
        if 'id' in source_df.columns:
            new_row['id'] = row['id']
        if fish_name_col in source_df.columns:
            new_row['name'] = row[fish_name_col]
        
        # 为每个结构体类型添加一对struct_type和coeff列
        for i, struct_type in enumerate(struct_columns):
            # 确定struct_type列和coeff列的名称
            type_col = f'struct_type{"" if i == 0 else f".{i}"}'  
            coeff_col = f'coeff{"" if i == 0 else f".{i}"}'
            
            # 将结构类型名称和系数值放入对应列
            new_row[type_col] = struct_type
            new_row[coeff_col] = float(row[struct_type])
        
        # 添加备注列 (如果存在)
        if 'mark' in source_df.columns:
            new_row['mark'] = row['mark']
        elif '备注' in source_df.columns:
            new_row['mark'] = row['备注']
        
        # 将新行添加到结果中
        result_rows.append(new_row)
    
    # 创建结果DataFrame
    result_df = pd.DataFrame(result_rows)
    return result_df

In [ ]:
# 将OriginDf中的水下结构体数据转换为目标格式并保存

# 1. 从设计表中提取水下结构体列
struct_columns = segmentStruct  # 使用之前提取的结构体列名
print("水下结构体列名:", struct_columns)

# 2. 转换为结构化格式
structured_df = convert_wide_to_structured(OriginDf, struct_columns)

# 3. 检查结果
print("\n转换后的前5行:")
print(structured_df.head())

# 4. 保存到Excel文件（可选）
output_path = os.path.join(rootPath, "intermediateTables", "StructuredEnvData.xlsx")
structured_df.to_excel(output_path, sheet_name='StructAffinity', index=False)
print(f"\n已保存结构化数据到: {output_path}")

水下结构体列名: ['[水下结构体]开放水域', '[水下结构体]水草', '[水下结构体]石头', '[水下结构体]沉木', '[水下结构体]桥墩']

转换后的前5行:
            name  struct_type  coeff struct_type.1  coeff.1 struct_type.2  \
0          Tench  [水下结构体]开放水域    0.0     [水下结构体]水草      0.0     [水下结构体]石头   
1   Golden_Bream  [水下结构体]开放水域    0.0     [水下结构体]水草      0.0     [水下结构体]石头   
2  Green_Sunfish  [水下结构体]开放水域    0.6     [水下结构体]水草      1.0     [水下结构体]石头   
3  Black_Crappie  [水下结构体]开放水域    0.6     [水下结构体]水草      1.0     [水下结构体]石头   
4  White_Crappie  [水下结构体]开放水域    0.6     [水下结构体]水草      1.0     [水下结构体]石头   

   coeff.2 struct_type.3  coeff.3 struct_type.4  coeff.4  \
0      0.0     [水下结构体]沉木      1.0     [水下结构体]桥墩      0.0   
1      0.0     [水下结构体]沉木      1.0     [水下结构体]桥墩      0.0   
2      0.6     [水下结构体]沉木      0.0     [水下结构体]桥墩      0.0   
3      0.6     [水下结构体]沉木      0.0     [水下结构体]桥墩      0.0   
4      0.6     [水下结构体]沉木      0.6     [水下结构体]桥墩      0.0   

                      mark  
0                      NaN  
1  在地图中间弄个特殊的地方放，并在水面上做点提示  
2 

In [ ]:
structSheet = AffinityPyxl['StructAffinity']

for r_idx, originRow in enumerate(dataframe_to_rows(structured_df, index=False, header=False)):
    targetRowName = "Cover_" + originRow[0]
    qualities = extract_qualities_for_fish(originRow[0])
    for q in qualities:
        rowInTopAffinitySheet = getFishNameRowFromTopAffinitySheet(originRow[0],q)
        print(f"RowInTopAffinitySheet : ", rowInTopAffinitySheet)
        if rowInTopAffinitySheet == -1:
            print(f"没有找到鱼种 {originRow[0]} 的行名")
            continue
        print(f"Row : ", originRow)

        topAffinitySheet.cell(row=rowInTopAffinitySheet, column=4, value=targetRowName)
    # print(targetRowName)
    headlessRow = originRow[1:-1] # 去掉行名和最后一列的备注
    # print(headlessRow)
    # 在第二列查找行名
    for cell in structSheet["B"]:
        if cell.value == targetRowName:
            # 找到行名，更新数据,更新的方法是将cell对应的行里，0、1元素不变，2（即第三个元素）号元素起，跟headlessRoww一样长的元素，使用headlessRow的值
            n = len(headlessRow)
            for i in range(n):
                structSheet.cell(row=cell.row, column=i+3, value=headlessRow[i])
            break
    else:
        # 如果没有找到行名，则添加新行, and keep the added row as a reference
        # 这里的row[1:]是去掉了行名的部分
        # 这一行的第一个数据是id，用上一行的id往上涨，涨到一个最近的能被10整除的数；
        # 也就是说，id是10的倍数
        maxRow = structSheet.max_row
        lastId = structSheet.cell(maxRow, 1).value
        if lastId is None:
            lastId = 2010000
        else:
            lastId = int(lastId)
        newId = lastId + 10 - (lastId % 10)
        # print(lastId)
        # add a new row, and keep the added row as a reference, the first cell is newId, the second cell is the row name, appended are headlessRow
        newRow = [newId, targetRowName] + list(headlessRow)
        structSheet.append(newRow)

AffinityPyxl.save(AffinitySpreadSheet)
AffinityPyxl.save("StructAffinity")
# AffinityPyxl.close()

RowInTopAffinitySheet :  126
Row :  ['Tench', '[水下结构体]开放水域', 0.0, '[水下结构体]水草', 0.0, '[水下结构体]石头', 0.0, '[水下结构体]沉木', 1.0, '[水下结构体]桥墩', 0.0, nan]
RowInTopAffinitySheet :  127
Row :  ['Golden_Bream', '[水下结构体]开放水域', 0.0, '[水下结构体]水草', 0.0, '[水下结构体]石头', 0.0, '[水下结构体]沉木', 1.0, '[水下结构体]桥墩', 0.0, '在地图中间弄个特殊的地方放，并在水面上做点提示']
RowInTopAffinitySheet :  75
Row :  ['Green_Sunfish', '[水下结构体]开放水域', 0.6, '[水下结构体]水草', 1.0, '[水下结构体]石头', 0.6, '[水下结构体]沉木', 0.0, '[水下结构体]桥墩', 0.0, nan]
RowInTopAffinitySheet :  53
Row :  ['Green_Sunfish', '[水下结构体]开放水域', 0.6, '[水下结构体]水草', 1.0, '[水下结构体]石头', 0.6, '[水下结构体]沉木', 0.0, '[水下结构体]桥墩', 0.0, nan]
RowInTopAffinitySheet :  54
Row :  ['Green_Sunfish', '[水下结构体]开放水域', 0.6, '[水下结构体]水草', 1.0, '[水下结构体]石头', 0.6, '[水下结构体]沉木', 0.0, '[水下结构体]桥墩', 0.0, nan]
RowInTopAffinitySheet :  80
Row :  ['Black_Crappie', '[水下结构体]开放水域', 0.6, '[水下结构体]水草', 1.0, '[水下结构体]石头', 0.6, '[水下结构体]沉木', 0.0, '[水下结构体]桥墩', 0.0, nan]
RowInTopAffinitySheet :  49
Row :  ['Black_Crappie', '[水下结构体]开放水域', 0.6, '[水下结构体]水草',

In [ ]:
FishBasicPyxl.save(FishBasicSpreadSheet)